In [197]:
using Revise, DynamicHMC, PosteriorDB, Random, StanLogDensityProblems, JSON, BridgeStan, DataFrames, LinearAlgebra, StatsBase, Distributions
using Plots, ColorSchemes, LaTeXStrings, Statistics, CSV, AdvancedHMC, DynamicObjects
Plots.theme(:default)
const BS = BridgeStan;

In [198]:
pdb = PosteriorDB.database()
model = PosteriorDB.model(pdb, "eight_schools_noncentered")
post = PosteriorDB.posterior(pdb, "eight_schools-eight_schools_noncentered")
impl = PosteriorDB.implementation(model, "stan")
path = PosteriorDB.path(impl)
data = PosteriorDB.dataset(post)
s = PosteriorDB.load(data, String);
bs_model = BS.StanModel(stan_file=path, data=s);
ref = PosteriorDB.reference_posterior(post)
df=DataFrame(PosteriorDB.load(ref));

┌ Warning: Loading a shared object '/home/meenaljhajharia/.julia/artifacts/95c3074bd791a2f28857088639b48ca1b786459a/posteriordb-0.4.0/posterior_database/models/stan/eight_schools_noncentered_model.so' which is already loaded.
│ If the file has changed since the last time it was loaded, this load may not update the library!
└ @ BridgeStan /home/meenaljhajharia/.julia/packages/BridgeStan/nCPwJ/src/model.jl:51


In [209]:
using TransformVariables
using TransformedLogDensities, LogDensityProblemsAD
t = as((μ = asℝ, σ = asℝ₊, τ = asℝ₊, θs = as(Array, 8)))
dimension(t)
lp = StanProblem(bs_model);
P = TransformedLogDensity(t, lp)
∇P = ADgradient(:ForwardDiff, lp);

D = 10;
initial_θ = rand(D);
n_samples, n_adapts = 8000, 2000;
metric = DiagEuclideanMetric(D);
hamiltonian = Hamiltonian(metric, lp);
initial_ϵ = find_good_stepsize(hamiltonian, initial_θ);
integrator = Leapfrog(initial_ϵ);
proposal = NUTS{MultinomialTS, GeneralisedNoUTurn}(integrator);
adaptor = StanHMCAdaptor(MassMatrixAdaptor(metric), StepSizeAdaptor(0.95, integrator));
samples, stats = sample(hamiltonian, proposal, initial_θ, n_samples, adaptor, n_adapts; progress=true);

Sampling  14%|████▍                          |  ETA: 0:00:01
  iterations:                                   1142
  ratio_divergent_transitions:                  0.0
  ratio_divergent_transitions_during_adaption:  0.01
  n_steps:                                      15
  is_accept:                                    true
  acceptance_rate:                              0.9678762233984499
  log_density:                                  -12.495620605415679
  hamiltonian_energy:                           18.573497841093214
  hamiltonian_energy_error:                     -0.22696809653136896
  max_hamiltonian_energy_error:                 -0.47517096939249726
  tree_depth:                                   4
  numerical_error:                              false
  step_size:                                    0.33844332860068677
  nom_step_size:                                0.33844332860068677
  is_adapt:                                     true



















Sampling  36%|███████████▏                   |  ETA: 0:00:00
  iterations:                                   2863
  ratio_divergent_transitions:                  0.0
  ratio_divergent_transitions_during_adaption:  0.0
  n_steps:                                      15
  is_accept:                                    true
  acceptance_rate:                              0.9862284275650175
  log_density:                                  -8.739985520249249
  hamiltonian_energy:                           12.334493859333428
  hamiltonian_energy_error:                     0.018817095252451566
  max_hamiltonian_energy_error:                 0.036422959906794006
  tree_depth:                                   4
  numerical_error:                              false
  step_size:                                    0.201554004050859
  nom_step_size:                                0.201554004050859
  is_adapt:                                     false



















Sampling  56%|█████████████████▌             |  ETA: 0:00:00
  iterations:                                   4509
  ratio_divergent_transitions:                  0.0
  ratio_divergent_transitions_during_adaption:  0.0
  n_steps:                                      15
  is_accept:                                    true
  acceptance_rate:                              0.9874697599563051
  log_density:                                  -8.529637180340215
  hamiltonian_energy:                           10.539590232234453
  hamiltonian_energy_error:                     0.04007651103199272
  max_hamiltonian_energy_error:                 -0.08424435540964659
  tree_depth:                                   4
  numerical_error:                              false
  step_size:                                    0.201554004050859
  nom_step_size:                                0.201554004050859
  is_adapt:                                     false



















Sampling  72%|██████████████████████▍        |  ETA: 0:00:00
  iterations:                                   5762
  ratio_divergent_transitions:                  0.0
  ratio_divergent_transitions_during_adaption:  0.0
  n_steps:                                      15
  is_accept:                                    true
  acceptance_rate:                              0.9659148930168432
  log_density:                                  -3.944477317232856
  hamiltonian_energy:                           10.0143479142909
  hamiltonian_energy_error:                     0.020435130838881932
  max_hamiltonian_energy_error:                 0.06491583894256259
  tree_depth:                                   4
  numerical_error:                              false
  step_size:                                    0.201554004050859
  nom_step_size:                                0.201554004050859
  is_adapt:                                     false



















Sampling  91%|████████████████████████████▍  |  ETA: 0:00:00
  iterations:                                   7309
  ratio_divergent_transitions:                  0.0
  ratio_divergent_transitions_during_adaption:  0.0
  n_steps:                                      31
  is_accept:                                    true
  acceptance_rate:                              0.9967164728705556
  log_density:                                  -9.177766186701803
  hamiltonian_energy:                           16.922118184660764
  hamiltonian_energy_error:                     -0.012175602045324752
  max_hamiltonian_energy_error:                 -0.03263173117069229
  tree_depth:                                   5
  numerical_error:                              false
  step_size:                                    0.201554004050859
  nom_step_size:                                0.201554004050859
  is_adapt:                                     false



















Sampling 100%|███████████████████████████████| Time: 0:00:00
  iterations:                                   8000
  ratio_divergent_transitions:                  0.0
  ratio_divergent_transitions_during_adaption:  0.0
  n_steps:                                      31
  is_accept:                                    true
  acceptance_rate:                              0.9855507414557796
  log_density:                                  -8.80501256393078
  hamiltonian_energy:                           15.438297910490672
  hamiltonian_energy_error:                     0.01727329036897096
  max_hamiltonian_energy_error:                 0.028640475254968933
  tree_depth:                                   5
  numerical_error:                              false
  step_size:                                    0.201554004050859
  nom_step_size:                                0.201554004050859
  is_adapt:                                     false
  mass_matrix:                    

┌ Info: Finished 8000 sampling steps for 1 chains in 0.627926098 (s)
│   h = Hamiltonian(metric=DiagEuclideanMetric([0.9645826069161378, 0.7092 ...]), kinetic=GaussianKinetic())
│   κ = HMCKernel{AdvancedHMC.FullMomentumRefreshment, Trajectory{MultinomialTS, Leapfrog{Float64}, GeneralisedNoUTurn{Float64}}}(AdvancedHMC.FullMomentumRefreshment(), Trajectory{MultinomialTS}(integrator=Leapfrog(ϵ=0.202), tc=GeneralisedNoUTurn{Float64}(10, 1000.0)))
│   EBFMI_est = 0.9696018535731714
│   average_acceptance_rate = 0.9754097102268455
└ @ AdvancedHMC /home/meenaljhajharia/.julia/packages/AdvancedHMC/P0nla/src/sampler.jl:246


In [210]:
constrained_draws = hcat([vcat(col...) for col in eachcol(df)]...)
unc_sample_array = vcat([
    param_unconstrain(bs_model, collect(row))' for row in eachrow(constrained_draws)
]...);

In [211]:
lol = hcat(samples...)'
conhmc = vcat([
    param_constrain(bs_model, collect(row))' for row in eachrow(lol)
]...);

In [212]:
mean(unc_sample_array, dims=1)

1×10 Matrix{Float64}:
 6.1505  4.93958  3.90591  4.79602  …  6.31717  4.884  4.41052  0.808081

In [213]:
mean(conhmc, dims=1)

1×10 Matrix{Float64}:
 0.320744  0.0992014  -0.0818857  0.0426832  …  0.0709181  4.33053  3.56593